# Stream Ciphers: Concepts, Implementation, and Cryptanalysis

## Introduction

Cryptography ensures secure communication in the digital age, and stream ciphers provide an efficient method for encrypting data streams in real-time applications. This essay explores the mechanics of stream ciphers, their implementation through Linear Feedback Shift Registers (LFSR), the cryptanalytic vulnerabilities revealed by the Berlekamp-Massey Algorithm, and the implications of Known-Plaintext Attacks (KPA). By examining their technical foundations and historical evolution, this analysis underscores the strengths and challenges of stream cipher cryptography (Stallings, 2017).



## Stream Ciphers


Originating with Gilbert Vernam’s One-Time Pad in 1917, which marked a milestone in information-theoretic security, stream ciphers treat plaintext as a stream, encrypting bits individually using the XOR operation for both encryption and decryption (Shannon, 1949). This makes them smaller and faster than block ciphers, ideal for applications like secure mobile communications. Security depends on the keystream, generated by random number generators (RNGs), which must be reproducible for decryption yet unpredictable to attackers. The One-Time Pad achieves perfect secrecy with a key as long as the plaintext, uniformly distributed, and used only once, but its impracticality due to key management has driven reliance on pseudorandom RNGs. Stream ciphers can achieve high secrecy with robust RNG designs (Katz & Lindell, 2014).

treats the plaintext as a stream and encrypts the bits individually

smaller and faster 

assumed to be more efficient

encryption and decryption are the same procedure -XOR

they can reach very high level of secrecy

One time pod: key as long as the plaintext, uniformly distributed in the key space, key must be used only once -> information theory-wise secure

Security in stream ciphers relies on key stream

OTP is unpractical because stream ciphers rely on random number generators

-maybe talk a bit about RNGs- 

RNG must be reproducible and unpredictable



In [1]:
from bits import Bits
from lfsr import LFSR, berlekamp_massey
from bitgenerator import AlternatingStep

In [2]:
random = [1, 0, 1, 0, 1, 0, 1, 0]
bits = Bits(random)
bits

Bits([True, False, True, False, True, False, True, False])

## LFSR

one of the main building blocks of PRNGs

In [3]:
lfsr = LFSR({1, 2, 5}, 0b001)
bits = lfsr.cycle()
bits

Bits([False, False, False, False, True, True, False, True, True, True, True, False, False, True])

In [4]:
lfsr = LFSR({1, 2, 5}, 0b00101)
output = lfsr.run_steps(10)

In [5]:
lfsr.__next__()

False

In [6]:
output

Bits([True, False, True, False, False, True, True, True, False, True])

In [7]:
bits = lfsr.cycle(state=0b00101)
bits

Bits([False, True, False, False, True, True, True])

## Berlekamp-Massey Algorithm

binary output sequence -> shortest lfsr able to produce the sequence

exploits the ptoperty that x p_i+b[t-i] must be zero

makes the system prone to KPA attck. If eve knows enough x_i and y_i's she can compute b_i's and apply B-M algorithm to infer P(X)

In [8]:
test_bits = Bits([1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1])
print(berlekamp_massey(test_bits))


{0, 3, 5}


In [9]:
with open('binary_sequence.bin', 'rb') as f:
        binary_sequence = f.read()

In [10]:
binary_sequence[:50]

b'\xbb`\xef\x067\xae\xd0K"Vd]#Q\xeb\x02~<\xe6C\xbe\xed5\xd0\xec\xada\xe8\x89h\xf3\xbdFc\x96\xb5\x8e\xb0\x03\xabVFY#\xd1\xeb">\xb5\xe4'

In [11]:
bits = Bits(binary_sequence)

In [12]:
poly = berlekamp_massey(bits)
linear_complexity = max(poly) if poly else 0

print("Shortest feedback polynomial degrees:", poly)
print("Linear complexity:", linear_complexity)

Shortest feedback polynomial degrees: {0, 18, 7}
Linear complexity: 18


## Alternating Step Generator

In [13]:
alt_step = AlternatingStep()

In [14]:
bits = alt_step.run(25)

In [15]:
bits

Bits([False, False, False, True, True, True, False, True, False, True, False, True, True, True, False, False, False, True, False, False, True, False, True, True, False])

# KPA

In [41]:
with open("ciphertext.bin", "rb") as f:
    ciphertext = f.read()

with open("known-plaintext.txt", "r", encoding="utf-8") as f:
    known_plaintext = f.read().encode("utf-8")

In [42]:
print(f"Ciphertext: {ciphertext[:30]} ..., \nKnown-Plaintext: {known_plaintext[:90]}...")

Ciphertext: b"\xb7;\xcep\x9e\x7f\xc0\xe3H_'\xc6D\x9b\xe8\xbd\x8e[\x8b\xb0\x94\x00\xdf]\xa9\xd9\x152k\x06" ..., 
Known-Plaintext: b'The Legacy of the Hidden Key\n\nIn a quiet corner of the university library, where dust mote'...


In [43]:
kp_bits = Bits(known_plaintext)
cipher_bits = Bits(ciphertext)

In [44]:
bit_sequence = kp_bits ^ Bits(cipher_bits[:len(kp_bits)])
print(f"Bit sequence: {bit_sequence[:10]} ...")

Bit sequence: [True, True, True, False, False, False, True, True, False, True] ...


In [45]:
poly = berlekamp_massey(bit_sequence)
linear_complexity = max(poly) if poly else 0

print("Shortest feedback polynomial degrees:", poly)
print("Linear complexity:", linear_complexity)

Shortest feedback polynomial degrees: {0, 1, 9, 48, 19}
Linear complexity: 48


In [46]:
init_state = Bits(bit_sequence[:48][::-1])
lfsr = LFSR(poly, init_state)

In [47]:
decryption_bits = lfsr.run_steps(len(cipher_bits), state=init_state)

In [49]:
decrypted_bits = decryption_bits ^ cipher_bits

In [55]:
decrypted_bytes = bytes(int(''.join(map(str, map(int, decrypted_bits[i:i+8]))), 2) for i in range(0, len(decrypted_bits), 8))
decrypted_bytes[:130]

b'The Legacy of the Hidden Key\n\nIn a quiet corner of the university library, where dust motes danced in the slanted afternoon light,'